In [1]:
# 기초 전처리
import pandas as pd
import numpy as np
import geopandas as gpd

# 컬럼 전체 확인 가능하도록 출력 범위 설정
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

# 불필요한 경고 표시 생략
import warnings
warnings.filterwarnings(action = 'ignore')

# pandas 결과값의 표현 범위 소수점 2자리수로 변경
pd.options.display.float_format = '{:.2f}'.format

# 파일 로드위한 directory 확인 및 현재 경로로 설정
import os
a = os.getcwd()
os.chdir(a)

C:\Users\HB\anaconda3\envs\my_env\lib\site-packages\geopandas\_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
df = pd.read_csv('서울시 우리마을가게 상권분석서비스(상권영역).csv', encoding='cp949')
# 좌표값 포함한 상권 영역 데이터 불러오기
bs_area = gpd.read_file('상권_geometry_좌표.geojson', driver= 'GeoJSON')
ref_code = pd.read_csv('행정동 및 법정동코드와의 연계표.csv', encoding = 'cp949')

In [3]:
# 이어 붙일 기본 파일
df_base = bs_area[['TRDAR_CD', 'TRDAR_SE_1','area']].sort_values('TRDAR_CD').reset_index(drop=True)
df_base.columns = ['상권_코드','상권_이름','상권_area']
df_base['상권_코드'] = df_base['상권_코드'].apply(lambda x : str(x))
df_base.astype( {'상권_코드':object} )

# 지리적으로 중복되는 상권 영역인 관광특구 제외
not_bs_area = df_base['상권_이름'] !='관광특구'
df_base = df_base[not_bs_area]

# 이어 붙일 기본 파일2
# 지리적으로 중복되는 상권 영역인 관광특구 제외
not_bs_area2 = df['상권_구분_코드_명'] !='관광특구'
df = df[not_bs_area2]

df = df[['상권_코드','행정동_코드']]
df['상권_코드'] = df['상권_코드'].apply(lambda x : str(x))
df['행정동_코드'] = df['행정동_코드'].apply(lambda x : str(x))

# 이어 붙일 기본 파일3
# 법정동에 시, 구 단위코드도 포함되어 493개 (동만 따지만 467개)
# ref_code['법정동'].str[-1].unique() => array(['시', '구', '동', '로', '가']
# 동단위만 찾기 위해 법정동 컬럼의 끝이 '동' 또는 '로' 또는 '가'로 된 데이터만 추림

ref_code = ref_code[ref_code['법정동'].str.endswith('동') | ref_code['법정동'].str.endswith('로') | ref_code['법정동'].str.endswith('가')]
ref_code = ref_code[['행정기관코드','법정동코드','행정동(행정기관명)']]
ref_code.columns = ['행정동_코드','법정동코드','행정동(행정기관명)']
ref_code['행정동_코드'] = ref_code['행정동_코드'].apply(lambda x : str(x))
ref_code['법정동코드'] = ref_code['법정동코드'].apply(lambda x : str(x))
ref_code['행정동_코드'] = ref_code['행정동_코드'].str[:8] # 코드 규격 맞추기 위해 끝 2자리('00') 제거

In [4]:
# 상권코드에 대응하는 행정동 코드 1490 rows
df_base = pd.merge(df_base, df, on='상권_코드')


df_base = pd.merge(df_base, ref_code, on='행정동_코드', how='left').sort_values('상권_코드')
df_base

,상권_코드,상권_이름,상권_area,행정동_코드,법정동코드,행정동(행정기관명)
0,1000001,골목상권,125350.78,11110600,1111014600,가회동
1,1000001,골목상권,125350.78,11110600,1111014700,가회동
2,1000001,골목상권,125350.78,11110600,1111014800,가회동
3,1000001,골목상권,125350.78,11110600,1111014900,가회동
4,1000002,골목상권,31696.16,11110710,1111017500,숭인제2동
...,...,...,...,...,...,...
3014,1001486,전통시장,10256.84,11740610,1174010900,천호제2동
3015,1001487,전통시장,28311.15,11740610,1174010900,천호제2동
3016,1001488,전통시장,23607.00,11740650,1174010800,성내제2동
3017,1001489,전통시장,36303.10,11740660,1174010800,성내제3동


- 행정동 범위 > 법정동 범위
- 상권은 행정&법정동 내부에 있을수도 혹은 다수의 행정&법정동에 걸쳐서 있을 수도 있다.
    - 현재 상권_코드가 1490개가 아니라 3019개인 이유

In [5]:
df_base.drop_duplicates('상권_코드')

,상권_코드,상권_이름,상권_area,행정동_코드,법정동코드,행정동(행정기관명)
0,1000001,골목상권,125350.78,11110600,1111014600,가회동
4,1000002,골목상권,31696.16,11110710,1111017500,숭인제2동
21,1000003,골목상권,113805.69,11110615,1111013700,종로1.2.3.4가동
38,1000004,골목상권,44023.15,11110650,1111017200,혜화동
40,1000005,골목상권,367064.09,11110550,1111018400,부암동
...,...,...,...,...,...,...
3014,1001486,전통시장,10256.84,11740610,1174010900,천호제2동
3015,1001487,전통시장,28311.15,11740610,1174010900,천호제2동
3016,1001488,전통시장,23607.00,11740650,1174010800,성내제2동
3017,1001489,전통시장,36303.10,11740660,1174010800,성내제3동


In [6]:
df_base.drop_duplicates('행정동_코드')

,상권_코드,상권_이름,상권_area,행정동_코드,법정동코드,행정동(행정기관명)
0,1000001,골목상권,125350.78,11110600,1111014600,가회동
4,1000002,골목상권,31696.16,11110710,1111017500,숭인제2동
21,1000003,골목상권,113805.69,11110615,1111013700,종로1.2.3.4가동
38,1000004,골목상권,44023.15,11110650,1111017200,혜화동
40,1000005,골목상권,367064.09,11110550,1111018400,부암동
...,...,...,...,...,...,...
2754,1001290,전통시장,7461.94,11170630,1117012900,이촌제1동
2835,1001357,전통시장,42823.76,11305645,1130510300,우이동
2838,1001359,전통시장,51918.32,11320512,1132010700,창제2동
2845,1001366,전통시장,4737.32,11350665,1135010500,상계3.4동


In [7]:
df_base.drop_duplicates('법정동코드')

,상권_코드,상권_이름,상권_area,행정동_코드,법정동코드,행정동(행정기관명)
0,1000001,골목상권,125350.78,11110600,1111014600,가회동
1,1000001,골목상권,125350.78,11110600,1111014700,가회동
2,1000001,골목상권,125350.78,11110600,1111014800,가회동
3,1000001,골목상권,125350.78,11110600,1111014900,가회동
4,1000002,골목상권,31696.16,11110710,1111017500,숭인제2동
...,...,...,...,...,...,...
2741,1001285,전통시장,14217.81,11140680,1114017100,중림동
2742,1001285,전통시장,14217.81,11140680,1114017200,중림동
2743,1001285,전통시장,14217.81,11140680,1114017300,중림동
2744,1001285,전통시장,14217.81,11140680,1114017400,중림동


In [8]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3019 entries, 0 to 3018
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   상권_코드       3019 non-null   object 
 1   상권_이름       3019 non-null   object 
 2   상권_area     3019 non-null   float64
 3   행정동_코드      3019 non-null   object 
 4   법정동코드       2993 non-null   object 
 5   행정동(행정기관명)  2993 non-null   object 
dtypes: float64(1), object(5)
memory usage: 165.1+ KB


- 상권영역에 해당되지 않는 법정동이 존재한다.